# A Notebook Preparing the MoMA Dataset Step-by-Step

In [1]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load data
moma = pd.read_csv('../data/moma/Artworks.csv')
moma.head()

,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,...,ThumbnailURL,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Seat Height (cm),Duration (sec.)
0,"Ferdinandsbrücke Project, Vienna, Austria (Ele...",Otto Wagner,6210,"(Austrian, 1841–1918)",(Austrian),(1841),(1918),(Male),1896,Ink and cut-and-pasted painted pages on paper,...,http://www.moma.org/media/W1siZiIsIjUyNzc3MCJd...,NaN,NaN,NaN,48.6000,NaN,NaN,168.9000,NaN,NaN
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,"(French, born 1944)",(French),(1944),(0),(Male),1987,Paint and colored pencil on print,...,http://www.moma.org/media/W1siZiIsIjUyNzM3NCJd...,NaN,NaN,NaN,40.6401,NaN,NaN,29.8451,NaN,NaN
2,"Villa near Vienna Project, Outside Vienna, Aus...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),1903,"Graphite, pen, color pencil, ink, and gouache ...",...,http://www.moma.org/media/W1siZiIsIjUyNzM3NSJd...,NaN,NaN,NaN,34.3000,NaN,NaN,31.8000,NaN,NaN
3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,"(French and Swiss, born Switzerland 1944)",(),(1944),(0),(Male),1980,Photographic reproduction with colored synthet...,...,http://www.moma.org/media/W1siZiIsIjUyNzQ3NCJd...,NaN,NaN,NaN,50.8000,NaN,NaN,50.8000,NaN,NaN
4,"Villa, project, outside Vienna, Austria, Exter...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),1903,"Graphite, color pencil, ink, and gouache on tr...",...,http://www.moma.org/media/W1siZiIsIjUyNzQ3NSJd...,NaN,NaN,NaN,38.4000,NaN,NaN,19.1000,NaN,NaN


In [3]:
# Have column names handy
moma.columns

Index(['Title', 'Artist', 'ConstituentID', 'ArtistBio', 'Nationality',
       'BeginDate', 'EndDate', 'Gender', 'Date', 'Medium', 'Dimensions',
       'CreditLine', 'AccessionNumber', 'Classification', 'Department',
       'DateAcquired', 'Cataloged', 'ObjectID', 'URL', 'ThumbnailURL',
       'Circumference (cm)', 'Depth (cm)', 'Diameter (cm)', 'Height (cm)',
       'Length (cm)', 'Weight (kg)', 'Width (cm)', 'Seat Height (cm)',
       'Duration (sec.)'],
      dtype='object')

## 0. Preparation

### 0.1. The Question and Approach

What I want to know is this:
- How old was the artwork (i.e., how long after the work was complete) when it was acquired by MoMA?
- If the artist was alive when a given work was acquired, how old was he/she?
- If the artist was deceased when a given work was acquired, how many years after his/her death did this event occur?

These are not currently features of the dataset, so I'll have to engineer them as follows
- `artwork_age`: `acquisition_year` - `completed_year`
- `artist_age`: `acquisition_year` - `birth_year`
- `years_posthumous`: `acquisition_year` - `death_year`

However, these aren't features either, but will have to be extracted/parsed from the original dataset:
- `acquisition_year`: parse from `DateAcquired`
- `completed_year`: parse and extract from `Date`
- `birth_year`: parse from `BeginDate`
- `death_year`: parse from `EndDate`

Thus, to simplify our dataset moving forward, I'll filter for the following features:
- `Title`
- `Artist`
- `BeginDate`
- `EndDate`
- `Date`
- `DateAcquired`

### 0.2. Dealing with Multiple Artist Records

Before jumping in, there's one additional issue, which is that sometimes a single record has more than one artist associated with it.

This makes sense in the case of collaborations, for example--presumably less common in certain mediums/classifications (e.g., painting, sculpture) than others (e.g., architecture, books).

Before continuing, let's get a better handle on how pervasive this issue is, and what kinds of artworks it tends to affect most. That way we'll be in a better position to know how to deal with them.

In [4]:
print("There are {:,} records in this dataset".format(len(moma)))

There are 140,848 records in this dataset


We can see evidence of multiple artists in the following features:
- Names in `Artist` are comma-separated
- Each bio in `ArtistBio` is contained in its own set of parentheses
- Each nationality in `Nationality` is contained in its own set of parentheses
- Each artist birth year in `BeginDate` is contained in its own set of parentheses
- Each artist death year in `EndDate` is contained in its own set of parentheses
- Each artist gender in `Gender` is contained in its own set of parentheses

I'll go through each of these and ensure that they are all pointing to the same number of multi-artist works. We want to drop these.

In [5]:
multi_artist = moma['Artist'].str.contains(',').fillna(False)
multi_artistbio = moma['ArtistBio'].str.match(r'.+?\).+?\(').fillna(False)
multi_begindate = moma['BeginDate'].str.match(r'.+?\).+?\(').fillna(False)
multi_enddate = moma['EndDate'].str.match(r'.+?\).+?\(').fillna(False)
multi_gender = moma['Gender'].str.match(r'.+?\).+?\(').fillna(False)
multi_nationality = moma['Nationality'].str.match(r'.+?\).+?\(').fillna(False)

print("`Artist` multiples: {:,}".format(len(moma[multi_artist])))
print("`ArtistBio` multiples: {:,}".format(len(moma[multi_artistbio])))
print("`BeginDate` multiples: {:,}".format(len(moma[multi_begindate])))
print("`EndDate` multiples: {:,}".format(len(moma[multi_enddate])))
print("`Gender` multiples: {:,}".format(len(moma[multi_gender])))
print("`Nationality` multiples: {:,}".format(len(moma[multi_nationality])))

`Artist` multiples: 8,291
`ArtistBio` multiples: 6,986
`BeginDate` multiples: 7,741
`EndDate` multiples: 7,741
`Gender` multiples: 7,741
`Nationality` multiples: 7,741


In [6]:
print("Equality of `multi_begindate` and `multi_enddate`: {}"
      .format(multi_begindate.equals(multi_enddate)))
print("Equality of `multi_begindate` and `multi_gender`: {}"
      .format(multi_begindate.equals(multi_gender)))
print("Equality of `multi_begindate` and `multi_nationality`: {}"
      .format(multi_begindate.equals(multi_nationality)))

Equality of `multi_begindate` and `multi_enddate`: True
Equality of `multi_begindate` and `multi_gender`: True
Equality of `multi_begindate` and `multi_nationality`: True


We can see that the indicators of multiple artists in `BeginDate`, `EndDate`, `Gender`, and `Nationality` are consistent, but I'm curious about the discrepancy with `Artist` indicators in particular.

In [7]:
moma[multi_artist & ~ multi_gender]

,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,...,ThumbnailURL,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Seat Height (cm),Duration (sec.)
902,"The Atheneum, New Harmony, Indiana","Richard Meier & Associates, Architects",22754,(founded 1963),(),(1963),(0),(),1975-79,Styrene,...,http://www.moma.org/media/W1siZiIsIjIyOTI2MSJd...,NaN,75.5652,NaN,24.800000,NaN,NaN,104.20,NaN,NaN
922,"Mediatheque, Sendai, Miyagi Prefecture, Japan ...","Toyo Ito & Associates, Architects",8987,"(Japan, established 1971)",(Japanese),(1971),(0),(),1995–2001,Acrylic,...,http://www.moma.org/media/W1siZiIsIjIxMTI5MyJd...,NaN,74.0000,NaN,27.000000,NaN,NaN,80.00,NaN,NaN
926,"Federal Building and United States Courthouse,...","Richard Meier & Partners, Architects",22753,(founded 1963),(),(1963),(0),(),1993-2000,Wood,...,http://www.moma.org/media/W1siZiIsIjIyOTU3NSJd...,NaN,66.7000,NaN,86.400000,NaN,NaN,127.70,NaN,NaN
944,"Shimosuwa Municipal Museum, Shimosuwa-machi, N...","Toyo Ito & Associates, Architects",8987,"(Japan, established 1971)",(Japanese),(1971),(0),(),1990–1993,Plexiglass and aluminum,...,http://www.moma.org/media/W1siZiIsIjUyNzY5MyJd...,NaN,60.0076,NaN,19.367539,NaN,NaN,120.00,NaN,NaN
969,Battery Jar,"Corning Glass Works, Corning, NY",1249,(est. 1851),(American),(1851),(0),(),1920s,Pyrex glass,...,NaN,NaN,NaN,30.4801,60.700000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134482,"56 Leonard Street, New York, New York, USA","Herzog & de Meuron, Basel",7567,(est. 1978),(Swiss),(1978),(0),(),2006–2008,Acrylic,...,NaN,NaN,5.0000,NaN,24.000000,NaN,NaN,4.00,NaN,NaN
134483,"56 Leonard Street, New York, New York, USA","Herzog & de Meuron, Basel",7567,(est. 1978),(Swiss),(1978),(0),(),2006–2008,Plexiglass,...,NaN,NaN,5.0000,NaN,30.000000,NaN,NaN,6.00,NaN,NaN
134484,"56 Leonard Street, New York, New York, USA","Herzog & de Meuron, Basel",7567,(est. 1978),(Swiss),(1978),(0),(),2006–2008,Acrylic,...,NaN,NaN,5.0000,NaN,30.000000,NaN,NaN,5.00,NaN,NaN
138641,"Untitled (Galeria de la Plaza, 613 N Main Stre...","Plaza Gallery, Los Angeles",133909,NaN,(),(0),(0),(),c. 1900,Collodion silver print,...,http://www.moma.org/media/W1siZiIsIjQ5OTQ5MiJd...,NaN,NaN,NaN,14.000000,NaN,NaN,9.70,NaN,NaN


The issue is commas in the `Artist` field, which are sometimes used to separate artist names but also sometimes parts of firm names, which would apply presumably to architecture and design.

In [8]:
moma[multi_artist & ~ multi_gender]['Classification'].value_counts()

Design              408
Architecture         74
Photograph           48
Print                14
Illustrated Book      2
Video                 2
Installation          1
(not assigned)        1
Name: Classification, dtype: int64

In [9]:
is_photograph =  (moma['Classification'] == 'Photograph')
moma[multi_artist & ~ multi_gender]['Artist'].value_counts()

Herzog & de Meuron, Basel                                                                67
Henry Wessel, Jr.                                                                        23
Coors Porcelain Co., Golden, CO                                                          22
Daum Frères, Nancy, France                                                               22
Department of Publications and Urban Design, Organizing Committee of the XIX Olympiad    19
                                                                                         ..
Hämmerli, Ltd., Lenzburg, Switzerland                                                     1
Slazengers Ltd., England                                                                  1
Inoue Pleats Co., Ltd., Fukui, Japan                                                      1
The Custanite Corp., Brooklyn, NY                                                         1
Van Cleave, Axtell, KS                                                          

Because indicators of multiple authorship are clearly less reliable/consistent in the `Artist` field, I'll ignore those in favor of the `Gender`/`BeginDate`/`EndDate`/`Nationality` indicators.

In [10]:
multi_artist = multi_gender

moma_solo = moma[~ multi_artist].copy()

print("{:,} total records of single-artist works".format(len(moma_solo)))

133,107 total records of single-artist works


## 1. Parsing Features and Extracting Relevant Data

Next is to parse these columns of interest and extract what we need.

### 1.1. Generating `year_acquired` from `DateAcquired`
Here we need to parse the `DateAcquired` feature and extract from it `year_acquired`

In [11]:
# Convert `DateAcquired` to datetime object and extract year
moma_solo['year_acquired'] = pd.to_datetime(moma_solo['DateAcquired']).dt.year.astype(float)

# Preview
moma_solo[['DateAcquired', 'year_acquired']].sample(20)

,DateAcquired,year_acquired
118357,2014-05-14,2014.0
78036,NaN,NaN
32829,1974-02-12,1974.0
61070,1992-05-20,1992.0
122714,2016-10-17,2016.0
22837,1964-10-06,1964.0
95176,2007-11-12,2007.0
120040,1991-12-09,1991.0
40500,1968-03-06,1968.0
22477,1964-10-06,1964.0


### 1.2. Parse `Date` Feature
The `Date` column holds the date attributed to the artwork, which seems like it should be straightforward enough but is actually a bit complicated because of artworks dated with a range or estimated date. So I'll extract both a `begun_year` feature as well as a `completed_year` feature, the latter being the thing we actually want for this analysis. As an intermediate step, I'll create a `date_stripped` feature which elimiates some of the extraneous details and transforms the value into a YYYY-YY or YYYY-YYYY format.

#### 1.2.1. Sampling Non-Standard Dates

To get to know the data better, I want to look for values that don't match an expected format, namely something standard (YYYY, preceded or followed by anything non-numerical) or hyphenated (YYYY-YY, YYYY-YYYY, or any variation that uses a hyphen, an n-dash, a slash, etc., and again preceded or followed by anything non-numerical).

I also want to look for values that are wholly non-numerical, since these are going to be NaN.

In [12]:
# Filter for non-numerical values
non_numerical = (moma_solo['Date'].str.match(r'^[^\d]+?$').fillna(False))

moma_solo[non_numerical]['Date'].value_counts()

n.d.                                                                                596
Unknown                                                                             239
(n.d.)                                                                              112
unknown                                                                              21
(London?, published in aid of the Comforts Fund  for Women and Children of Sovie     10
no date                                                                               4
n.d                                                                                   3
TBC                                                                                   3
New York                                                                              2
TBD                                                                                   2
Various                                                                               1
Unkown                          

In [13]:
# Filter for standard format dates
standard_format = (moma_solo['Date'].str.match(r'^.*?\d{4}[^\d]*?$').fillna(False))

moma_solo[standard_format].sample(10)

,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,...,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Seat Height (cm),Duration (sec.),year_acquired
23596,"Plate 16 from L'avaguardia internazionale, Vol...",Mimmo Rotella,5041,"(Italian, 1918–2006)",(Italian),(1918),(2006),(Male),1962,Etching,...,NaN,NaN,NaN,14.8000,NaN,NaN,11.3000,NaN,NaN,1964.0
59647,Woodblock for Fishing Boats (Fischerboote),Lyonel Feininger,1832,"(American, 1871–1956)",(American),(1871),(1956),(Male),1918,Woodblock,...,NaN,0.9,NaN,12.4000,NaN,NaN,16.2000,NaN,NaN,1966.0
44411,Ann Pennington,Edward Steichen,5623,"(American, born Luxembourg. 1879–1973)",(American),(1879),(1973),(Male),1925,Gelatin silver print,...,NaN,NaN,NaN,34.9251,NaN,NaN,27.3051,NaN,NaN,1961.0
105823,Department of Eagles (David–Ingres–Wiertz–Cour...,Marcel Broodthaers,795,"(Belgian, 1924–1976)",(Belgian),(1924),(1976),(Male),1968,Painted vacuum-formed plastic plate,...,NaN,0.5,NaN,84.0000,NaN,NaN,121.2000,NaN,NaN,2011.0
126483,"Positive Ambiguity (beard, lectern, teleprompt...",DIS,47179,"(American, founded 2010)",(American),(2010),(0),(),2015,"Video (color, sound)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,166.0,2016.0
98714,Divertevents One,Frederic Lieberman,38023,"(American, 1930–2013)",(American),(1930),(2013),(Male),c. 1965,"Envelope, containing nine offset cards",...,NaN,0.0,NaN,8.8000,NaN,NaN,11.4000,NaN,NaN,2008.0
53242,"Liberty, with French, Indian, Highlander, and ...",Lee Friedlander,2002,"(American, born 1934)",(American),(1934),(0),(Male),1975,Gelatin silver print,...,NaN,NaN,NaN,18.8000,NaN,NaN,28.1000,NaN,NaN,2000.0
33902,Study for Metermaid,Pat Andrea,175,"(Dutch, born 1942)",(Dutch),(1942),(0),(Male),1968,"Colored pencil, pencil, and crayon on paper",...,NaN,NaN,NaN,50.1651,NaN,NaN,65.0876,NaN,NaN,1972.0
51344,New York City,Lee Friedlander,2002,"(American, born 1934)",(American),(1934),(0),(Male),1979,Gelatin silver print,...,NaN,NaN,NaN,47.1000,NaN,NaN,31.3000,NaN,NaN,1994.0
64771,Scene from Tevia the Milkman by Sholem Aleichem,Anatoli Lvovich Kaplan,2991,"(Russian, 1902–1980)",(Russian),(1902),(1980),(Male),(1961),Lithograph,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1961.0


In [14]:
# Filter for hyphenated format dates
hyphenated_format = (moma_solo['Date'].str.match(r'^[^\d]*?\d{4} ?[-–/] ?\d{2,4}[^\d]*?$').fillna(False))

moma_solo[hyphenated_format].sample(10)

,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,...,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Seat Height (cm),Duration (sec.),year_acquired
23896,Plate (folio 5) from LA MÉTROMANIE,Jean Dubuffet,1633,"(French, 1901–1985)",(French),(1901),(1985),(Male),1949-1950,NaN,...,NaN,NaN,NaN,20.000000,NaN,NaN,19.700000,NaN,NaN,1964.0
80044,"Josef Esters House, Krefeld, Germany, Plan, up...",Ludwig Mies van der Rohe,7166,"(American, born Germany. 1886–1969)",(American),(1886),(1969),(Male),c. 1927-30,Pencil on tracing paper,...,NaN,NaN,NaN,69.900000,NaN,NaN,95.900000,NaN,NaN,NaN
89694,Chair with Arms. Frame Detail. (Elevation sket...,Ludwig Mies van der Rohe,7166,"(American, born Germany. 1886–1969)",(American),(1886),(1969),(Male),1926-46,Pencil on paper,...,NaN,NaN,NaN,32.702600,NaN,NaN,20.955000,NaN,NaN,1974.0
88595,Untitled (1) from Untitled,Carroll Dunham,1652,"(American, born 1949)",(American),(1949),(0),(Male),2004-06,One from a series of fourteen lithographs,...,NaN,NaN,NaN,47.200000,NaN,NaN,57.300000,NaN,NaN,2006.0
27385,Headpiece (page 148) from FLORILÈGE DES AMOURS,Henri Matisse,3832,"(French, 1869–1954)",(French),(1869),(1954),(Male),1941-1948,NaN,...,NaN,NaN,NaN,9.600000,NaN,NaN,9.600000,NaN,NaN,1964.0
78831,Sketchbook,Paul Delvaux,1485,"(Belgian, 1897–1994)",(Belgian),(1897),(1994),(Male),(c. 1939-44),Pen and ink and wash on paper,...,NaN,NaN,NaN,13.000000,NaN,NaN,18.400000,NaN,NaN,1964.0
114686,Hubbe-related Study,Ludwig Mies van der Rohe,7166,"(American, born Germany. 1886–1969)",(American),(1886),(1969),(Male),1934-1935,Ink on paper,...,NaN,NaN,NaN,20.955042,NaN,NaN,29.210058,NaN,NaN,NaN
31630,Man Standing,Gaston Lachaise,3332,"(American, born France. 1882–1935)",(American),(1882),(1935),(Male),c. 1928–29,Pencil on paper,...,NaN,NaN,NaN,61.000000,NaN,NaN,48.300000,NaN,NaN,1935.0
32733,Untitled from Cuban Sketch Book,Jules Pascin,4514,"(American, born Bulgaria. 1885–1930)",(American),(1885),(1930),(Male),(c. 1914-20),Conté crayon on paper,...,NaN,NaN,NaN,10.800000,NaN,NaN,18.700000,NaN,NaN,1935.0
40066,Pont Neuf,Eugène Atget,229,"(French, 1857–1927)",(French),(1857),(1927),(Male),1910-11,Gelatin silver printing-out-paper print,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1968.0


Now let's sample the `Date` feature for values that are non-standard and non-hyphenated, but still have numbers in there somewhere.

In [15]:
non_standard = (~hyphenated_format & ~standard_format &~non_numerical)

print("There are {:,} non-standard `Date` values."
      .format(len(moma_solo[non_standard])))

print()

print(moma_solo[non_standard]['Date'].dropna().sample(20))

There are 2,073 non-standard `Date` values.

132003                     1964, assembled c. 1965-66
71901                         1947, published 1952–53
68915                   1962, prints executed 1958-60
62487                        (1922, executed 1920-21)
108935    1893 (reproduced drawings executed 1891–93)
131846                        1963, assembled 1964–65
132017                     1961, assembled c. 1965-66
13876                  (1950, print executed 1949-50)
65893                    (1920-21, published 1922-23)
62489                        (1922, executed 1920-21)
13869                  (1950, print executed 1949-50)
75314                     1967 (refabricated 1973-75)
110642                                 (c. 1910s-30s)
108955    1893 (reproduced drawings executed 1891–93)
132022                     1961, assembled c. 1965-66
92565                      1915–16, published 1916–17
110637                                 (c. 1910s-30s)
13873                  (1950, print e

These could be dealt with individually, since there are clear patterns here. But since there are so few of these kinds of formats (1-2% of the dataset in my estimation), I'm going to be a little faster and looser with extracting dates.

#### 1.2.2. Extracting Stripped Dates

In [16]:
# Simplify `Date`
moma_solo['date_stripped'] = moma_solo['Date'].str.extract(r'^.*?(\d{4} ?[-–/]? ?\d{0,4})')

moma_solo[['Date', 'date_stripped']].sample(10)

,Date,date_stripped
135525,July 1962,1962
114649,1934-1935,1934-1935
80812,1991,1991
91476,1966,1966
21904,1941-1948,1941-1948
77481,2000,2000
136173,1946,1946
109307,C. 1990-1999,1990-1999
16971,1966-1970,1966-1970
25956,1919,1919


That's working as expected, but let's see what's happening to the non-standard examples from above:

In [17]:
moma_solo[non_standard][['Date', 'date_stripped']].dropna().sample(10)

,Date,date_stripped
13863,"(1950, prints executed 1949-50)",1950
68407,"1959, printed 1963–64",1959
13879,"(1950, print executed 1949-50)",1950
26222,(c. 1940s-60s),1940
77383,1920s–30s,1920
131907,"1958, assembled 1964–65",1958
68833,"1960, prints executed 1958-59",1960
132034,"1962, assembled c. 1965-66",1962
13880,"(1950, print executed 1949-50)",1950
96457,"1961, realized 1963-64",1961


This is a compromise I'm willing to live with. We're losing a little bit of granularity and specificity, since we're ending up only with the date a work was conceived but not the date it was produced, or vice-versa. But we'll still end up with a good approximation--fine for our purposes here.

#### 1.2.3. Extracting `begun_year` and `completed_year` from `date_stripped`
With a cleaner field to work with, we can now extract start and end dates from each artwork.

In [18]:
# Extract start year from `date_stripped
moma_solo['begun_year'] = moma_solo['date_stripped'].str[:4].astype(float)

# Extract end year from `date_stripped
moma_solo['completed_year'] = (
    moma_solo['date_stripped'].str.extract(r'.*(\d{2})\d{2}') 
    + moma_solo['date_stripped'].str.extract(r'(\d{2})[-–/]?$')
).astype(float)

# Preview results
moma_solo[['Date', 'date_stripped', 'begun_year', 'completed_year']].sample(20, random_state=123)

,Date,date_stripped,begun_year,completed_year
121803,1962,1962,1962.0,1962.0
86326,NaN,NaN,NaN,NaN
49187,1948,1948,1948.0,1948.0
117585,1973,1973,1973.0,1973.0
18770,"1923, published 1977",1923,1923.0,1923.0
42425,1932,1932,1932.0,1932.0
18250,"1941, published 1943",1941,1941.0,1941.0
115693,1916-23,1916-23,1916.0,1923.0
33947,1912,1912,1912.0,1912.0
65814,1947,1947,1947.0,1947.0


And again, let's just have a look at the non-standard ones:

In [19]:
moma_solo[non_standard][['Date', 'date_stripped', 'begun_year', 'completed_year']].dropna().sample(10)

,Date,date_stripped,begun_year,completed_year
135799,1870s-80s,1870,1870.0,1870.0
131906,"1958, assembled 1964–65",1958,1958.0,1958.0
132028,"1962, assembled c. 1965-66",1962,1962.0,1962.0
97161,"1958, printed 1962-63",1958,1958.0,1958.0
131902,"1958, assembled 1964–65",1958,1958.0,1958.0
73536,1911? (dated on painting 1911-12),1911,1911.0,1911.0
13871,"(1950, print executed 1949-50)",1950,1950.0,1950.0
135803,1870s-80s,1870,1870.0,1870.0
62492,"(1922, executed 1920-21)",1922,1922.0,1922.0
132053,"1958, assembled c. 1965-66",1958,1958.0,1958.0


### 1.3. Parsing `BeginDate`/`EndDate` and Extracting `birth_year`/`death_year`

Next is to deal with artist birth year and death year

In [20]:
# Create new features for artist birth year and death year
moma_solo['birth_year'] = moma_solo['BeginDate'].str.extract(r'\((\d+?)\)', ).astype(float)
moma_solo['death_year'] = moma_solo['EndDate'].str.extract(r'\((\d+?)\)').astype(float)

# Preview
moma_solo[['BeginDate', 'birth_year', 'EndDate', 'death_year']].sample(20)

,BeginDate,birth_year,EndDate,death_year
25742,(1885),1885.0,(1954),1954.0
94601,(1953),1953.0,(0),0.0
128616,(1877),1877.0,(1966),1966.0
82269,(1837),1837.0,(1921),1921.0
92123,(1928),1928.0,(1984),1984.0
33745,(1940),1940.0,(0),0.0
103693,(1978),1978.0,(0),0.0
33899,(1934),1934.0,(0),0.0
69848,(1884),1884.0,(1976),1976.0
18063,(1896),1896.0,(1987),1987.0


Make sure that there are no weird birth years.

In [21]:
moma_solo['birth_year'].sort_values().unique()

array([   0., 1730., 1731., 1746., 1753., 1765., 1772., 1782., 1787.,
       1789., 1792., 1795., 1796., 1797., 1798., 1799., 1800., 1801.,
       1802., 1804., 1808., 1809., 1810., 1811., 1812., 1813., 1814.,
       1815., 1816., 1817., 1818., 1819., 1820., 1821., 1822., 1823.,
       1824., 1825., 1826., 1827., 1828., 1829., 1830., 1831., 1832.,
       1833., 1834., 1835., 1836., 1837., 1838., 1839., 1840., 1841.,
       1842., 1843., 1844., 1845., 1846., 1847., 1848., 1849., 1850.,
       1851., 1852., 1853., 1854., 1855., 1856., 1857., 1858., 1859.,
       1860., 1861., 1862., 1863., 1864., 1865., 1866., 1867., 1868.,
       1869., 1870., 1871., 1872., 1873., 1874., 1875., 1876., 1877.,
       1878., 1879., 1880., 1881., 1882., 1883., 1884., 1885., 1886.,
       1887., 1888., 1889., 1890., 1891., 1892., 1893., 1894., 1895.,
       1896., 1897., 1898., 1899., 1900., 1901., 1902., 1903., 1904.,
       1905., 1906., 1907., 1908., 1909., 1910., 1911., 1912., 1913.,
       1914., 1915.,

Replace 0 with NaN

In [22]:
null_birthyear = (moma_solo['birth_year'] == 0)

moma_solo.loc[null_birthyear, 'birth_year'] = np.nan

### 1.4. Review
Here's how are dataset now looks, focusing on columns of interest:

In [23]:
cols = [
    'Title', 'Artist', 'BeginDate', 'EndDate', 'Date', 'DateAcquired', 
    'year_acquired', 'date_stripped', 'begun_year', 'completed_year', 
    'birth_year', 'death_year'
]

moma_solo[cols].sample(10)

,Title,Artist,BeginDate,EndDate,Date,DateAcquired,year_acquired,date_stripped,begun_year,completed_year,birth_year,death_year
50538,Young Couple,Juichi Nagano,(0),(0),c. 1950,1959-04-22,1959.0,1950,1950.0,1950.0,NaN,0.0
73198,The labor agent who had been sent South by Nor...,Jacob Lawrence,(1917),(2000),1940-41,1942-01-27,1942.0,1940-41,1940.0,1941.0,1917.0,2000.0
100936,Green File Boxes,NaN,NaN,NaN,NaN,2008-10-08,2008.0,NaN,NaN,NaN,NaN,NaN
90518,Untitled from Gymnasium Chases,Christian Boltanski,(1944),(2021),1991 (project begun 1989).,1992-01-22,1992.0,1991,1991.0,NaN,1944.0,2021.0
104116,"Belfast City Center, Belfast",Gilles Peress,(1946),(0),1972,2011-05-18,2011.0,1972,1972.0,1972.0,1946.0,0.0
23422,Headpiece (page 81) from POÈMES,Charles Despiau,(1874),(1946),1933,1964-10-06,1964.0,1933,1933.0,1933.0,1874.0,1946.0
108857,Painting,Isa Genzken,(1948),(0),1989,2011-12-12,2011.0,1989,1989.0,1989.0,1948.0,0.0
137987,Cóctel (Cocktail),Alejandro Kuropatwa,(1956),(2003),1996,2020-10-07,2020.0,1996,1996.0,1996.0,1956.0,2003.0
87399,Kytes Tree,Gabriel Orozco,(1962),(0),2005,2005-12-12,2005.0,2005,2005.0,2005.0,1962.0,0.0
23919,In-text plate (folio 7 verso) from LA MÉTROMANIE,Jean Dubuffet,(1901),(1985),1949-1950,1964-10-06,1964.0,1949-1950,1949.0,1950.0,1901.0,1985.0


## 2. Computing New Features

Now we're ready to compute the remaining features we need
- `artwork_age`: `acquisition_year` - `completed_year`
- `artist_age`: `acquisition_year` - `birth_year`
- `years_posthumous`: `acquisition_year` - `death_year`

### 2.1. Compute `artwork_age` for all works

In [24]:
moma_solo['artwork_age'] = moma_solo['year_acquired'] - moma_solo['completed_year']

# Preview
moma_solo[['completed_year', 'year_acquired', 'artwork_age']].sample(10, random_state=111)

,completed_year,year_acquired,artwork_age
94488,1930.0,NaN,NaN
51741,1976.0,2000.0,24.0
41005,1922.0,1968.0,46.0
50725,1955.0,1998.0,43.0
22006,1933.0,1964.0,31.0
37951,1925.0,1968.0,43.0
105652,1996.0,2011.0,15.0
29527,1930.0,1964.0,34.0
14045,1911.0,1966.0,55.0
89781,1932.0,1974.0,42.0


### 2.2. Engineer `living` feature to categorize whether artist was alive or deceased at acquisition

In [25]:
moma_solo['living'] = (
    np.where((moma_solo['year_acquired'] < moma_solo['death_year']) | (moma_solo['death_year'] == 0), 1, 0)
)

# Preview
moma_solo[['year_acquired', 'death_year', 'living']].sample(10, random_state=111)

,year_acquired,death_year,living
94488,NaN,1969.0,0
51741,2000.0,0.0,1
41005,1968.0,1927.0,0
50725,1998.0,1984.0,0
22006,1964.0,1979.0,1
37951,1968.0,1927.0,0
105652,2011.0,0.0,1
29527,1964.0,1974.0,1
14045,1966.0,1957.0,0
89781,1974.0,1969.0,0


### 2.3. Compute `artist_age` for Artists Alive at Acquisition

In [26]:
moma_solo['artist_age'] = (
    np.where(moma_solo['living'] == 1, moma_solo['year_acquired'] - moma_solo['birth_year'], np.nan)
)

# Preview
moma_solo[['year_acquired', 'birth_year', 'death_year', 'living', 'artist_age']].sample(10, random_state=111)

,year_acquired,birth_year,death_year,living,artist_age
94488,NaN,1886.0,1969.0,0,NaN
51741,2000.0,1934.0,0.0,1,66.0
41005,1968.0,1857.0,1927.0,0,NaN
50725,1998.0,1928.0,1984.0,0,NaN
22006,1964.0,1898.0,1979.0,1,66.0
37951,1968.0,1857.0,1927.0,0,NaN
105652,2011.0,1946.0,0.0,1,65.0
29527,1964.0,1884.0,1974.0,1,80.0
14045,1966.0,1871.0,1957.0,0,NaN
89781,1974.0,1886.0,1969.0,0,NaN


### 2.4. Compute `years_posthumous` for Artists Deceased at Acquisition

In [27]:
moma_solo['years_posthumous'] = (
    np.where(moma_solo['living'] == 0, moma_solo['year_acquired'] - moma_solo['death_year'], np.nan)
)

# Preview
moma_solo[['year_acquired', 'birth_year', 'death_year', 'living', 'artist_age', 'years_posthumous']].sample(10, random_state=111)

,year_acquired,birth_year,death_year,living,artist_age,years_posthumous
94488,NaN,1886.0,1969.0,0,NaN,NaN
51741,2000.0,1934.0,0.0,1,66.0,NaN
41005,1968.0,1857.0,1927.0,0,NaN,41.0
50725,1998.0,1928.0,1984.0,0,NaN,14.0
22006,1964.0,1898.0,1979.0,1,66.0,NaN
37951,1968.0,1857.0,1927.0,0,NaN,41.0
105652,2011.0,1946.0,0.0,1,65.0,NaN
29527,1964.0,1884.0,1974.0,1,80.0,NaN
14045,1966.0,1871.0,1957.0,0,NaN,9.0
89781,1974.0,1886.0,1969.0,0,NaN,5.0


Looking good!

## 3. Test Cleaning Script

I've incorporated all the above steps into `art_stats_utils.py`. Now I want to confirm that the results are identical.

In [28]:
from art_stats_utils import prepare_dataset

df = pd.read_csv('../data/moma/Artworks.csv')

df = prepare_dataset(df)

In [29]:
df.equals(moma_solo)

True

All set!